In [1]:
import pandas as pd

#Load in all the different csv files
compas_path = "data/compas"
recidivism_path = "data/recidivism"

#Compas data
compas_scores_raw = pd.read_csv(f"{compas_path}/compas-scores-raw.csv")
compas_scores_two_years = pd.read_csv(f"{compas_path}/compas-scores-two-years.csv")
compas_database = sqlite3.connect(f"{compas_path}/compas.db")
#tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", compas_database)

#Recidivism data for other states
iowa_recidivism = pd.read_csv(f"{recidivism_path}/Current_Iowa_Correctional_System_Prison_Population_.csv")

In [30]:
#Compas_scores_raw
##Checking number of unique people
#number_of_unique_people = compas_scores_raw["Person_ID"].unique().shape[0]   
#number_of_unique_people 


compas_scores_raw["Agency_Text"].unique()



array(['PRETRIAL', 'Probation', 'DRRD', 'Broward County'], dtype=object)

##### Compas Scores Raw:
There are a total of 18610 unique Person_IDs with 20281 unique AssesmentIDs. Hence, there must be people being assesed more than once. There are also 19530 case IDs


In [ ]:
###COMPAS SCORES TWO YEARS#############
print(compas_scores_two_years.dtypes)
#print first 2 values for each column
for attribute in compas_scores_two_years.columns:
    print(f"{attribute}: {compas_scores_two_years[attribute][0]}")


In [40]:
######IOWA RECIDIVISM#############
print(iowa_recidivism.dtypes)
#print first 2 values for each column
for attribute in iowa_recidivism.columns:
    print(f"{attribute}: {iowa_recidivism[attribute][0]}")

Record ID                          int64
Offender Number                    int64
Prison Start Date                 object
Prison Admission Type             object
Intake Supervising Institution    object
Supervision Status                object
Sex                               object
Race & Ethnicity                  object
Education Category                object
Education                         object
Age                                int64
Offense Code                      object
Offense Classification            object
Offense Description               object
Offense Type                      object
Offense Subtype                   object
Months Served                      int64
Report Date                       object
dtype: object
Record ID: 0
Offender Number: 20167742
Prison Start Date: 05/18/2023
Prison Admission Type: Return No New Sentence
Intake Supervising Institution: North Central Correctional Facility
Supervision Status: Prison
Sex: Male
Race & Ethnicity: White
Educ

In [ ]:
import graphviz as gr
import numpy as np

Z1 = "sex"
Z2 = "age"
X = "race"
D = "c_charge_degree"
Y = "score_text"
P = "priors_count"
J_1 = "juv_fel_count"
J_2 = "juv_misd_count"
J_3 = "juv_other_count" 

#subset data from compas_scores_2_years
compas_scores_two_years_subset = compas_scores_two_years[[X, Z1, Z2, D, Y, P, J_1, J_2, J_3]]
compas_scores_two_years_subset["J"] =np.sum(compas_scores_two_years_subset[[J_1, J_2, J_3]], axis=1)
compas_scores_two_years_subset.drop([J_1, J_2, J_3], axis=1, inplace=True)

#print unique values for race 
#print(compas_scores_two_years_subset[X].unique())
#print(compas_scores_two_years[X].unique())
#print(compas_scores_raw["Ethnic_Code_Text"].unique())

#change 'Caucassian' to 'White' and all other values to 'Non-White'
compas_scores_two_years_subset[X] = compas_scores_two_years_subset[X].replace('Caucasian', 'White')
compas_scores_two_years_subset[X] = compas_scores_two_years_subset[X].replace(['African-American', 'Hispanic', 'Asian', 'Other', 'Native American'], 'Non-White')
print(compas_scores_two_years_subset[X].unique())

#Create a directed acyclic graph
DAG = gr.Digraph()
#Nodes
DAG.node(X) #node for X = Race
DAG.node(Z1) #node for Z1 = Gender
DAG.node(Z2) #node for Z2 = Age
DAG.node(D) #node for D = Degree of charge
DAG.node(Y) #node for Y = risk classification
DAG.node(P) #node for P = number of prior offenses
DAG.node("priors_J_count") #node for J = number of prior juvenile offenses
#Edges (directed)
DAG.edge(X, Y) #edge from X to Y
DAG.edge(Z1, Y) #edge from Z1 to Y
DAG.edge(Z2, Y) #edge from Z2 to Y
DAG.edge(D, Y) #edge from D to Y
DAG.edge("priors_J_count", Y) #edge from J to Y
DAG.edge(Z1, "priors_J_count") #edge from Z1 to J
DAG.edge(Z2, "priors_J_count") #edge from Z2 to J
DAG.edge(Z1,P) #edge from Z1 to P
DAG.edge(Z2,P) #edge from Z2 to P
DAG.edge(Z1, D) #edge from Z1 to D
DAG.edge(Z2, D) #edge from Z2 to D
DAG.edge(X, D) #edge from X to D
DAG.edge(X, P) #edge from X to P
DAG.edge(X,"priors_J_count") #edge from X to J
DAG.edge(Z1,Y) #edge from Z1 to Y
DAG.edge(Z2,Y) #edge from Z2 to Y
#Edges (dashed, bidirectional)
DAG.edge(X, Z1, style="dashed" , dir="both") #edge from X to Z1
DAG.edge(X, Z2, style="dashed", dir="both") #edge from X to Z2

#print the graph
display(DAG)

